In [9]:
import os,sys
print(sys.executable)
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS
from pyspark.sql.types import *
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit







spark = SparkSession \
    .builder \
    .appName("Spark ML App") \
     .getOrCreate()

schema = StructType([ \
    StructField("userId",  IntegerType()), \
    StructField("movieId", IntegerType()), \
    StructField("rating", FloatType())])

ratings = spark.read.format("csv")\
.options(header='false') \
.option("delimiter","\\t") \
.schema(schema) \
.load("resources/sample_movies_users.data")
print(ratings.describe().toPandas().transpose())

(training, test) = ratings.randomSplit([0.8, 0.2])
als = ALS(rank=10, maxIter=10,userCol='userId', itemCol='movieId', ratingCol='rating', regParam=0.1, coldStartStrategy="drop")
alsModel = als.fit(training)

predictions = alsModel.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))



testDF = spark.createDataFrame(
     [(0, 50, -1), (0, 172, -1), (0, 133, -1)],
   ["userId", "movieId", "rating"])
predictionDF=alsModel.transform(testDF);







spark.stop()



/opt/conda/bin/python
              0                   1                   2    3     4
summary   count                mean              stddev  min   max
userId   100003   462.4708758737238  266.62245405929286    0   943
movieId  100003  425.52094437166886   330.7977655087594    1  1682
rating   100003   3.529834104976851  1.1257051368972655  1.0   5.0
Root-mean-square error = 0.9185707624952067
